<h1>House price regression</h1>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder # mã hóa dữ liệu category
from sklearn.compose import ColumnTransformer # transform cột
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error # đánh giá độ chính xác
# chọn đặc trưng
from sklearn.feature_selection import SelectKBest, VarianceThreshold, SelectFromModel, mutual_info_regression

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
train_path = "train.csv"
test_path = "test.csv"

In [4]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [5]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.00000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.00000,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.00000,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.00000,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.00000,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.00000,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [8]:
print(f'Training dataset {train.shape} \n Testing dataset {test.shape}')

Training dataset (1460, 81) 
 Testing dataset (1459, 80)


<h2>Basic data exploration - Missing value

In [9]:
# Devide columns is category and numerical ones to have a better view
cat_columns = train.select_dtypes(include='object').columns.tolist()
num_columns = train.select_dtypes(exclude='object').columns.tolist()
print(f'Category Columns: \n{cat_columns} \n Numerical Columns: \n{num_columns}')

Category Columns: 
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'] 
 Numerical Columns: 
['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'Ga

In [10]:
# Remove last element (SalePrice) of train dataset
num_columns = num_columns[:-1]
num_columns

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

<h4>Missing values categorical features

In [11]:
train[cat_columns].isnull().sum()

MSZoning            0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinType2       38
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu       690
GarageType         81
GarageFinish       81
GarageQual         81
GarageCond         81
PavedDrive          0
PoolQC           1453
Fence            1179
MiscFeature      1406
SaleType            0
SaleCondition       0
dtype: int64

In [13]:
def check_missing(df):
    missing = df.isna().sum()[df.isna().any()==True]
    df_out = pd.DataFrame({'missing': missing})
    return df_out

check_missing(train[cat_columns])


,missing
Alley,1369
MasVnrType,8
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
Electrical,1
FireplaceQu,690
GarageType,81


In [14]:
check_missing(train[num_columns])

,missing
LotFrontage,259
MasVnrArea,8
GarageYrBlt,81


<h4>Splitting the dataset into Train & Test data

In [15]:
X = train.drop(['SalePrice'], axis=1)
y = train['SalePrice']

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=8)
x_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
884,885,20,RL,65.00000,7150,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdWo,NaN,0,7,2009,WD,Normal
641,642,60,FV,NaN,7050,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
1357,1358,20,RL,NaN,12537,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
469,470,60,RL,76.00000,9291,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal
222,223,60,RL,85.00000,11475,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,2,2006,WD,Normal


In [17]:
x_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1260,1261,60,RL,NaN,24682,Pave,NaN,IR3,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
274,275,20,RL,76.00000,8314,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2007,WD,Normal
51,52,50,RM,52.00000,6240,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,400,9,2006,WD,Normal
117,118,20,RL,74.00000,8536,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2007,New,Partial
789,790,60,RL,NaN,12205,Pave,NaN,IR1,Low,AllPub,...,0,0,NaN,NaN,NaN,0,7,2007,WD,Normal


In [18]:
y_train.head()

884     100000
641     226000
1357    149900
469     187000
222     179900
Name: SalePrice, dtype: int64

In [19]:
y_test.head()

1260    181000
274     124500
51      114500
117     155000
789     187500
Name: SalePrice, dtype: int64

<h3>Preprocessing - imputing - Encoding scaling data<h3>
<p>Dealing with the Numerical Features - imputing missing value

In [20]:
# make_pipeline: 
num_imputing = make_pipeline(
    SimpleImputer(strategy='constant', fill_value=0) # replace missing values by 0
)

<h4>Dealing with categorical features - imputing - ordinal encoding and one hot encoding

In [21]:
cat_imputing = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='NA') # replace missing values by NA
)

<h3>Standard data

In [22]:
ordinal_features = ['ExterQual','ExterCond','KitchenQual','BsmtQual','BsmtCond','HeatingQC','FireplaceQu','GarageQual',
'GarageCond','GarageFinish','BsmtExposure','BsmtFinType1','BsmtFinType2','Functional','CentralAir','LandSlope','PavedDrive',
'Fence','PoolQC','Alley','Street','Utilities']

nominal_features = list(set(cat_columns) - set(ordinal_features))
len(nominal_features)

21

In [43]:
len(ordinal_features)

22

In [ ]:
cat_columns

In [ ]:
nominal_features

In [26]:
# lists with values for ordinal feature encoding
ql5 =['None','Po','Fa','TA','Gd','Ex']
fin=['None','Unf','RFn','Fin']
expo=['None','No','Mn','Av','Gd']
fint=['None','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
func=['None','Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ']
yn=['Y','N']
ls=['None','Sev','Mod','Gtl']
pad=['N','P','Y']
fen=['None','MnWw','GdWo','MnPrv','GdPrv']
ql4=['None','Fa','TA','Gd','Ex']
al=['None','Grvl','Pave']
st=['None','Grvl','Pave']
util=['ELO','NoSeWa','NoSewr','AllPub']

# for each feature in the ordinal_feature list there has to be an encoding category in the following categories list to be passed to the OrdinalEncoder
ordinal_categories = [ql5,ql5,ql5,ql5,ql5,ql5,ql5,ql5,ql5,fin,expo,fint,fint,func,yn,ls,pad,fen,ql4,al,st,util]

In [44]:
len(ordinal_categories)

22

In [ ]:
ordinal_categories

In [29]:
len(ordinal_features)

22

In [30]:
len(num_columns)

37

In [31]:
len(ordinal_features) + len(nominal_features) + len(num_columns)
# len(ordinal_features) + len(nominal_features) = len(cat_columns)

80

In [32]:
# pipeline: mã hóa dữ liệu
ordinal_enc = Pipeline(steps=[
    ('ordinal_encoder', OrdinalEncoder(categories=ordinal_features))
])

one_hot_enc = Pipeline(steps=[
    ('one_hot_encoder', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [ ]:
ordinal_enc

In [ ]:
one_hot_enc

<h3>Combine Imputing and Encoding into the Pipeline

In [36]:
imputing = ColumnTransformer(transformers= [ 
    ('imp_nums', num_imputing, num_columns),
    ('imp_cats', cat_imputing, cat_columns)
])

encoding = ColumnTransformer(transformers=[ 
    ('enc_num', "passthrough", num_columns),
    ('enc_ord', ordinal_enc, ordinal_features),
    ('enc_nom', one_hot_enc, nominal_features)
]),


In [ ]:
imputing

In [ ]:
encoding

<h3>Scaling data

In [40]:
scaling = Pipeline(steps=[ 
    ('scale', MinMaxScaler())
])

<h3>Full Preprocess of the dataset

In [41]:
cat_encoding = ColumnTransformer(transformers=[ 
    ('enc_ord', ordinal_enc, ordinal_features),
    ('enc_nom', one_hot_enc, nominal_features)
])

cats = Pipeline(steps=[ 
    ('impute_cats', cat_imputing),
    ('encode_cats', one_hot_enc)
])

nums = Pipeline(steps=[ 
    ('impute_nums', num_imputing)
])

preprocess = ColumnTransformer(transformers=[ 
    ('cats', cats, cat_columns),
    ('nums', nums, num_columns)
])

full_preprocess2 = Pipeline(steps=[ 
    ('preprocess', preprocess),
    ('scaling', scaling)
])


In [45]:
pd.DataFrame(full_preprocess2.fit_transform(x_train))

,0,1,2,3,4,5,6,7,8,9,...,295,296,297,298,299,300,301,302,303,304
0,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,...,0.20310,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.54545,0.75000
1,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,...,0.45839,0.00000,0.44933,0.00000,0.00000,0.00000,0.00000,0.00000,0.36364,0.25000
2,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,...,0.35261,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.27273,1.00000
3,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,...,0.35684,0.19565,0.13384,0.00000,0.00000,0.00000,0.00000,0.00000,0.45455,0.50000
4,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,...,0.30606,0.28397,0.39771,0.00000,0.00000,0.00000,0.00000,0.00000,0.09091,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,1.00000,0.00000,...,0.17983,0.53533,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.45455,0.00000
1164,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,...,0.33216,0.26087,0.04780,0.00000,0.00000,0.00000,0.00000,0.00000,0.45455,0.75000
1165,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,...,0.16925,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.54545,0.50000
1166,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,1.00000,...,0.33850,0.00000,0.31740,0.00000,0.00000,0.00000,0.00000,0.00000,0.27273,0.00000


<h3>Model prediction - Simple Linear regression

In [46]:
lm_pipeline = Pipeline(steps=[ 
    ('full_preprocess', full_preprocess2),
    ('model', LinearRegression())
])


In [47]:
lm_pipeline.fit(x_train, y_train)

Pipeline(steps=[('full_preprocess',
                 Pipeline(steps=[('preprocess',
                                  ColumnTransformer(transformers=[('cats',
                                                                   Pipeline(steps=[('impute_cats',
                                                                                    Pipeline(steps=[('simpleimputer',
                                                                                                     SimpleImputer(fill_value='NA',
                                                                                                                   strategy='constant'))])),
                                                                                   ('encode_cats',
                                                                                    Pipeline(steps=[('one_hot_encoder',
                                                                                                     OneHotEncoder(handle_unknown='i

In [48]:
predictions = lm_pipeline.predict(x_test)

In [49]:
rmse = mean_squared_error(y_test, predictions)** 0.5
rmse

3588815091190252.0

In [50]:
rmsle = mean_squared_log_error(y_test, abs(predictions)) ** 0.5
rmsle

2.161382443599179

In [64]:
# evaluate the result
results = pd.DataFrame({
    'predictions': predictions,
    'true_values': y_test
})

results['diff'] = abs(round(results['predictions'] - results['true_values']))

results.sort_values(by='diff', ascending=True).head()

,predictions,true_values,diff
1155,217984.00000,218000,16.00000
1307,137984.00000,138000,16.00000
1189,188928.00000,189000,72.00000
933,189824.00000,190000,176.00000
1114,117248.00000,117000,248.00000


In [52]:
results.sort_values(by='diff', ascending=True).tail()

,predictions,true_values,diff
774,289280.00000,395000,105720.00000
197,480512.00000,235000,245512.00000
523,604416.00000,184750,419666.00000
346,-36528313519534720.00000,151500,36528313519686224.00000
332,-49259757296762624.00000,284000,49259757297046624.00000


<p>2 dòng cuối lệch nhiều là outliers, nên bỏ outliers

<h3>Features selection

In [68]:
select_vt = Pipeline(steps=[ 
    ('variance_threshold', VarianceThreshold(threshold=(0.9*(1-0.9))))
])

vt_pipeline = Pipeline(steps=[ 
    ('full_preprocess', full_preprocess2),
    ('feature_selection', select_vt),
    ('model', LinearRegression())
])

In [69]:
vt_pipeline.fit(x_train, y_train)

Pipeline(steps=[('full_preprocess',
                 Pipeline(steps=[('preprocess',
                                  ColumnTransformer(transformers=[('cats',
                                                                   Pipeline(steps=[('impute_cats',
                                                                                    Pipeline(steps=[('simpleimputer',
                                                                                                     SimpleImputer(fill_value='NA',
                                                                                                                   strategy='constant'))])),
                                                                                   ('encode_cats',
                                                                                    Pipeline(steps=[('one_hot_encoder',
                                                                                                     OneHotEncoder(handle_unknown='i

In [70]:
vt_predictions = vt_pipeline.predict(x_test)

In [71]:
rmsle = mean_squared_log_error(y_test, abs(vt_predictions))** 0.5
rmsle

0.23882672378480077

In [76]:
# evaluate the result
results1 = pd.DataFrame({
    'vt_predictions': vt_predictions,
    'true_values': y_test
})

results1['diff'] = abs(round(results1['vt_predictions'] - results1['true_values']))

results1.sort_values(by='diff', ascending=True).head()

,vt_predictions,true_values,diff
207,141543.68470,141000,544.00000
1257,99117.14821,99900,783.00000
1299,154898.99074,154000,899.00000
876,133165.92741,132250,916.00000
966,158939.47186,160000,1061.00000


In [65]:
results.sort_values(by='diff', ascending=True).tail()

,predictions,true_values,diff
774,289280.00000,395000,105720.00000
197,480512.00000,235000,245512.00000
523,604416.00000,184750,419666.00000
346,-36528313519534720.00000,151500,36528313519686224.00000
332,-49259757296762624.00000,284000,49259757297046624.00000
